In [ ]:
import pandas as pd
import numpy as np

In [2]:
DATA_PATH = "./data/"
MED_FILE = "PRESCRIPTIONS.csv"
DIAGS_FILE = "DIAGNOSES_ICD.csv"
NDC_TO_ADC_FILE = "package_NDC_ATC4_classes.csv"

In [ ]:
def process_med():
    Field = ["SUBJECT_ID","HADM_ID","ICUSTAY_ID","STARTDATE","NDC"]
    FieldType = {"SUBJECT_ID": 'Int64',
                "HADM_ID": 'Int64',
                "ICUSTAY_ID": 'Int64',
                "NDC": 'str',
                }
    med_df = pd.read_csv(DATA_PATH+MED_FILE, usecols=Field, dtype=FieldType, parse_dates=["STARTDATE"])

    med_df = med_df[med_df['NDC'] != '0'] 
    med_df.fillna(method='pad', inplace=True)
    med_df.dropna(inplace=True) 
    med_df.drop_duplicates(inplace=True)
    med_df.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE'], inplace=True)
    med_df = med_df.reset_index(drop=True)

    def filter_first24hour_med(med_df):
        med_df_new = med_df.drop(columns=['NDC'])
        med_df_new = med_df_new.groupby(by=['SUBJECT_ID','HADM_ID','ICUSTAY_ID']).head(1).reset_index(drop=True) #returns the first startdate
        med_df_new = pd.merge(med_df_new, med_df, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTDATE'])
        med_df_new = med_df_new.drop(columns=['STARTDATE'])
        return med_df_new
    med_df = filter_first24hour_med(med_df) 
    med_df = med_df.drop(columns=['ICUSTAY_ID'])
    med_df = med_df.drop_duplicates()

    return med_df.reset_index(drop=True)

In [9]:
med_df = process_med()
med_df["LenNDC"] = med_df['NDC'].str.len()
med_df.head(10)

C:\Users\kevin\AppData\Local\Temp\ipykernel_58968\1164149481.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  med_df.fillna(method='pad', inplace=True)


,SUBJECT_ID,HADM_ID,NDC,LenNDC
0,2,163353,63323017302,11
1,2,163353,63323038810,11
2,4,185777,00088222033,11
3,4,185777,00456068801,11
4,4,185777,61787006204,11
5,4,185777,00024159601,11
6,4,185777,00002735501,11
7,4,185777,00074407532,11
8,4,185777,58177000104,11
9,4,185777,00004006850,11


In [11]:
print(f"max is {med_df["LenNDC"].max()}, min is {med_df["LenNDC"].min()}")

max is 11, min is 11


In [5]:
Field = ["NDC","ATC_class"]
FieldType = {"NDC": 'str',
            "ATC_class": 'str'
            }
ndc2atc_df = pd.read_csv(DATA_PATH+NDC_TO_ADC_FILE, usecols=Field, dtype=FieldType)
ndc2atc_df['NDC'] = ndc2atc_df['NDC'].str.replace('-', '')
med_df = med_df.merge(ndc2atc_df, left_on='NDC', right_on='NDC')

In [ ]:
ndc2atc_df.dtypes

In [7]:
ndc2atc_df.head()

,NDC,ATC_class
0,5041942208,G02BA
1,6362989791,A01AD
2,0573016025,M01AE
3,5334500401,S01GA
4,2504775201,R02AD
